In [ ]:
!pip install transformers==3.0.2
!pip install nlp

In [ ]:
import gc
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from nlp import load_dataset
from transformers import TFAutoModel, AutoTokenizer

import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(123)

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
MODEL = 'jplu/tf-xlm-roberta-large'

EPOCHS = 4
MAXLEN = 120

BATCH_SIZE = 16 * strategy.num_replicas_in_sync

tokenizer = AutoTokenizer.from_pretrained(MODEL)
auto = tf.data.experimental.AUTOTUNE

In [ ]:
def create_model(transformer_layer,learning_rate):
    input_ids = Input(shape = (MAXLEN,), dtype = tf.int32)
    #input_masks = Input(shape = (MAXLEN,), dtype = tf.int32)

    #insert roberta layer
    roberta = TFAutoModel.from_pretrained(transformer_layer)
    roberta = roberta([input_ids])[0]

    out = GlobalAveragePooling1D()(roberta)

    #add our softmax layer
    out = Dense(3, activation = 'softmax')(out)

    #assemble model and compile
    model = Model(inputs = input_ids, outputs = out)
    model.compile(
                    optimizer = Adam(lr = learning_rate), 
                    loss = 'sparse_categorical_crossentropy', 
                    metrics = ['accuracy'])
    return model


def tokeniZer(text,tokenizer):
    # tokenize
    encoded = tokenizer.batch_encode_plus(text, padding=True, max_length=MAXLEN, truncation=True)
    #return np.array(encoded['input_ids'], encoded['attention_mask'])
    return np.array(encoded['input_ids'])

def tokeniZer_old(dataset,tokenizer):
    # tokenize
    text = dataset[['premise', 'hypothesis']].values.tolist()
    encoded = tokenizer.batch_encode_plus(text, padding=True, max_length=MAXLEN, truncation=True)
    # features
    x = encoded['input_ids'], encoded['attention_mask']
    # labels
    y = None
    if 'label' in dataset.columns:
        y = dataset.label.values
    return encoded['input_ids'], encoded['attention_mask'], y


def load_mnli(use_validation=True):
    result = []
    dataset = load_dataset(path='glue', name='mnli')
    print(dataset['train'])
    keys = ['train', 'validation_matched','validation_mismatched'] if use_validation else ['train']
    for k in keys:
        for record in dataset[k]:
            c1, c2, c3 = record['premise'], record['hypothesis'], record['label']
            if c1 and c2 and c3 in {0,1,2}:
                result.append((c1,c2,c3,'en'))
    result = pd.DataFrame(result, columns=['premise','hypothesis','label','lang_abv'])
    return result

def load_snli(use_validation=True):
    result = []
    dataset = load_dataset(path='snli')
    keys = ['train', 'validation'] if use_validation else ['train']
    for k in keys:
        for record in dataset[k]:
            c1, c2, c3 = record['premise'], record['hypothesis'], record['label']
            if c1 and c2 and c3 in {0,1,2}:
                result.append((c1,c2,c3,'en'))
    result = pd.DataFrame(result, columns=['premise','hypothesis','label','lang_abv'])
    return result

def load_xnli():
    result = []
    dataset = load_dataset(path='xnli')
    for k in dataset.keys():
        for record in dataset[k]:
            hp, pr, lb = record['hypothesis'], record['premise'], record['label']
            if hp and pr and lb in {0,1,2}:
                for lang, translation in zip(hp['language'], hp['translation']):
                    pr_lang = pr.get(lang, None)
                    if pr_lang is None:
                        continue
                    result.append((pr_lang, translation, lb,lang))
    result = pd.DataFrame(result, columns=['premise','hypothesis','label','lang_abv'])
    return result

In [ ]:
# load data
train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
test = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')
submission = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/sample_submission.csv')

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
mnli = load_mnli()

In [ ]:
mnli.shape

In [ ]:
mnli.head()

In [ ]:
total_train = train.drop(columns=['id'])
total_train = pd.concat([total_train, mnli], axis=0)

shuffled_data = shuffle(total_train).reset_index(drop = True)

X, y = shuffled_data[['premise', 'hypothesis']].values.tolist(), shuffled_data['label']

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=2020)

In [ ]:
x_train = tokeniZer(x_train,tokenizer)
x_valid = tokeniZer(x_valid,tokenizer)
x_test  = tokeniZer(test[['premise', 'hypothesis']].values.tolist(),tokenizer)

del mnli
gc.collect()

In [ ]:
# datasets
def build_dataset(x, y, mode, batch_size):
    if mode == "train":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices((x, y))
            .repeat()
            .shuffle(2048)
            .batch(batch_size)
            .prefetch(auto)
        )
    elif mode == "valid":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices((x, y))
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(auto)
        )
    elif mode == "test":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices(x)
            .batch(BATCH_SIZE)
        )
    else:
        raise NotImplementedError
    return dataset

In [ ]:
train_dataset = build_dataset(x_train, y_train, "train", BATCH_SIZE)
valid_dataset = build_dataset(x_valid, y_valid, "valid", BATCH_SIZE)
test_dataset  = build_dataset(x_test, None, "test", BATCH_SIZE)

In [ ]:
def create_xlm(transformer_layer,random_seed,learning_rate):
    tf.keras.backend.clear_session()
    tf.random.set_seed(random_seed)
    with strategy.scope():
        model = create_model(transformer_layer,learning_rate)
    model.summary()    
    return model

In [ ]:
Xlm = create_xlm(MODEL,1124234,1e-5)

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience = 2, 
                                              monitor = 'val_loss', 
                                              restore_best_weights = True, 
                                              mode = 'min')]

steps_per_epoch = len(x_train) // BATCH_SIZE
history_xlm = Xlm.fit(train_dataset,
                      validation_data=valid_dataset,
                      steps_per_epoch=steps_per_epoch,
                      epochs = EPOCHS, 
                      callbacks = callbacks)



In [ ]:
plt.plot(history_xlm.history['accuracy'])
plt.plot(history_xlm.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_xlm.history['loss'])
plt.plot(history_xlm.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#model predictions
predictions_xlm = Xlm.predict(test_dataset)
predictions = predictions_xlm
final = np.argmax(predictions, axis = 1)    

submission = pd.DataFrame()    
submission['id'] = test['id']
submission['prediction'] = final.astype(np.int32)

submission.to_csv('submission.csv', index = False)

In [ ]:
submission.head()